### Initialize notebook variables

Configures everything that's needed for deployment. 

**Modify entries under _1) User-defined parameters_ and _2) Define the APIs and their operations and policies_**.

In [ ]:
import utils
from apimtypes import *

# 1) User-defined parameters (change these as needed)
rg_location = 'eastus2'
index       = 1
deployment  = INFRASTRUCTURE.SIMPLE_APIM
tags        = ['authX', 'jwt', 'hr']       # ENTER DESCRIPTIVE TAG(S)
api_prefix  = 'authX-'                     # OPTIONAL: ENTER A PREFIX FOR THE APIS TO REDUCE COLLISION POTENTIAL WITH OTHER SAMPLES

# 2) Define the APIs and their operations and policies

# Create the notebook helper with JWT support
nb_helper = utils.NotebookHelper('authX', utils.get_infra_rg_name(deployment, index), rg_location, deployment, [INFRASTRUCTURE.AFD_APIM_PE, INFRASTRUCTURE.APIM_ACA, INFRASTRUCTURE.SIMPLE_APIM], use_jwt = True, index = index)

# Policies
# Named values must be set up a bit differently as they need to have two surrounding curly braces
pol_hr_all_operations = utils.read_policy_xml('hr_all_operations.xml', sample_name = 'authX').format(
    jwt_signing_key = '{{' + nb_helper.jwt_key_name + '}}', 
    hr_member_role_id = '{{HRMemberRoleId}}'
)
pol_hr_get = utils.read_policy_xml('hr_get.xml', sample_name = 'authX').format(
    hr_administrator_role_id = '{{HRAdministratorRoleId}}',
    hr_associate_role_id = '{{HRAssociateRoleId}}'
)
pol_hr_post = utils.read_policy_xml('hr_post.xml', sample_name = 'authX').format(
    hr_administrator_role_id = '{{HRAdministratorRoleId}}'
)

# API 1: Employees (HR)
hr_employees_get = GET_APIOperation('Gets the employees', pol_hr_get)
hr_employees_post = POST_APIOperation('Creates a new employee', pol_hr_post)
hr_employees = API('Employees', 'Employees', '/employees', 'This is a Human Resources API to obtain employee information', pol_hr_all_operations, operations = [hr_employees_get, hr_employees_post], tags = tags, subscriptionRequired = True)

# APIs Array
apis: List[API] = [hr_employees]

# Set up the named values
nvs: List[NamedValue] = [
    NamedValue(nb_helper.jwt_key_name, nb_helper.jwt_key_value_bytes_b64, True),
    NamedValue('HRMemberRoleId', Role.HR_MEMBER),
    NamedValue('HRAssociateRoleId', Role.HR_ASSOCIATE),
    NamedValue('HRAdministratorRoleId', Role.HR_ADMINISTRATOR)
]

utils.print_ok('Notebook initialized')

### Create deployment using Bicep

Creates the bicep deployment into the previously-specified resource group. A bicep parameters file will be created prior to execution.

In [ ]:
# Build the bicep parameters
bicep_parameters = {
    'apis': {'value': [api.to_dict() for api in apis]},
    'namedValues': {'value': [nv.to_dict() for nv in nvs]}
}

# Deploy the sample
output = nb_helper.deploy_sample(bicep_parameters)

if output.success:
    # Extract deployment outputs for testing
    apim_name        = output.get('apimServiceName', 'APIM Service Name')
    apim_gateway_url = output.get('apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_apis        = output.getJson('apiOutputs', 'APIs')

    print(f"✅ Sample deployment completed successfully!")
else:
    print("❌ Sample deployment failed!")
    raise SystemExit(1)

### Verify API Request Success

Assert that the deployment was successful by making simple calls to APIM. 

❗️ If the infrastructure shields APIM and requires a different ingress (e.g. Azure Front Door), the request to the APIM gateway URl will fail by design. Obtain the Front Door endpoint hostname and try that instead.

In [ ]:
from apimrequests import ApimRequests
from apimtesting import ApimTesting
from apimtypes import Role
from users import UserHelper
from authfactory import AuthFactory

# Initialize testing framework
tests = ApimTesting("AuthX Sample Tests", 'authX', nb_helper.deployment)
hr_api_apim_subscription_key = apim_apis[0]['subscriptionPrimaryKey']

# Check infrastructure endpoint
utils.print_message('Checking infrastructure endpoint...', blank_above = True)
afd_endpoint_url = utils.get_frontdoor_url(nb_helper.deployment, nb_helper.rg_name)
endpoint_url = afd_endpoint_url if afd_endpoint_url else apim_gateway_url

# 1) HR Administrator - Full access
encoded_jwt_token_hr_admin = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ADMINISTRATOR), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Admin:\n{encoded_jwt_token_hr_admin}')

reqsApimAdmin = ApimRequests(endpoint_url, hr_api_apim_subscription_key)
reqsApimAdmin.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_admin}'

output = reqsApimAdmin.singleGet('/employees', msg = 'Calling GET Employees API as HR Admin. Expect 200.')
tests.verify(output, 'Returning a mock employee')

output = reqsApimAdmin.singlePost('/employees', msg = 'Calling POST Employees API as HR Admin. Expect 200.')
tests.verify(output, 'A mock employee has been created.')

# 2) HR Associate - Read-only access
encoded_jwt_token_hr_associate = AuthFactory.create_symmetric_jwt_token_for_user(UserHelper.get_user_by_role(Role.HR_ASSOCIATE), nb_helper.jwt_key_value)
print(f'\nJWT token for HR Associate:\n{encoded_jwt_token_hr_associate}')

reqsApimAssociate = ApimRequests(endpoint_url, hr_api_apim_subscription_key)
reqsApimAssociate.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_associate}'

output = reqsApimAssociate.singleGet('/employees', msg = 'Calling GET Employees API as HR Associate. Expect 200.')
tests.verify(output, 'Returning a mock employee')

output = reqsApimAssociate.singlePost('/employees', msg = 'Calling POST Employees API as HR Associate. Expect 403.')
tests.verify(output, '')

# 3) Missing API subscription key
reqsNoApiSubscription = ApimRequests(endpoint_url)
reqsNoApiSubscription.headers['Authorization'] = f'Bearer {encoded_jwt_token_hr_admin}'

output = reqsNoApiSubscription.singleGet('/employees', msg = 'Calling GET Employees API without API subscription key. Expect 401.')
outputJson = utils.get_json(output)
tests.verify(outputJson['statusCode'], 401)
tests.verify(outputJson['message'], 'Access denied due to missing subscription key. Make sure to include subscription key when making requests to an API.')

tests.print_summary()
utils.print_ok('All done!')